In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
import time
import io
from PIL import Image
import random
from tensorflow.keras.models import load_model
import numpy as np

from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
with open('../user-agents.txt', 'r') as f:
    user_agents = f.read().split('\n')
print('User agents list number:', len(user_agents))

User agents list number: 1001


In [ ]:
def modify_link(link):
    if '[' in link and ']' in link:
        return link[: link.find('[')] + link[link.find('[') + 1: link.find(']')] + '-Version' + link[link.find(']') + 1:]
    return link

In [3]:
model = load_model('noise_classifier.h5')

In [4]:
# No car name: Freightliner

car_names = """Acura
Alfa Romeo
Audi
BMW
Buick
Cadillac
Chevrolet
Chrysler
Dodge
Fiat
Ford
Genesis
GMC
Honda
Hyundai
Infiniti
Jaguar
Jeep
Kia
Land Rover
Lexus
Lincoln
Maserati
Mazda
Mercedes-Benz
Mini
Mitsubishi
Nissan
Porsche
Ram
Subaru
Tesla
Toyota
Volkswagen
Volvo
Ferrari
Lamborghini
Rolls-Royce
Lotus
Aston Martin
Bentley
Maybach
Peugeot
Hummer
Bugatti"""

car_names = car_names.split('\n')
print(car_names)

['Acura', 'Alfa Romeo', 'Audi', 'BMW', 'Buick', 'Cadillac', 'Chevrolet', 'Chrysler', 'Dodge', 'Fiat', 'Ford', 'Genesis', 'GMC', 'Honda', 'Hyundai', 'Infiniti', 'Jaguar', 'Jeep', 'Kia', 'Land Rover', 'Lexus', 'Lincoln', 'Maserati', 'Mazda', 'Mercedes-Benz', 'Mini', 'Mitsubishi', 'Nissan', 'Porsche', 'Ram', 'Subaru', 'Tesla', 'Toyota', 'Volkswagen', 'Volvo', 'Ferrari', 'Lamborghini', 'Rolls-Royce', 'Lotus', 'Aston Martin', 'Bentley', 'Maybach', 'Peugeot', 'Hummer', 'Bugatti']


In [22]:
http_proxy  = "http://209.205.212.34:1203"
https_proxy = "https://209.205.212.34:1203"
ftp_proxy   = "ftp://209.205.212.34:1203"

proxyDict = { 
              "http"  : http_proxy, 
              "https" : https_proxy, 
              "ftp"   : ftp_proxy
            }

PROXY = "http://209.205.212.34:1203"

if not os.path.exists('Cars'):
    os.mkdir('Cars')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)
# chrome_options.add_argument('headless')

driver = webdriver.Chrome('../../chromedriver', chrome_options=chrome_options)
driver.maximize_window()

while True:
    driver.get('https://www.netcarshow.com/')
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, f"//*[contains(text(), '{car_names[-1]}')]")))
        print('Innitial page is found')
        break
    except:
        print('Wait Timeout for Innitial page, trying again.')
        continue


all_make_links = []
for make_name in car_names:
    all_make_links.append((make_name, driver.find_element_by_xpath(f"//*[contains(text(), '{make_name}')]").get_attribute('href')))
# driver.close()
    
for make_name, make_link in all_make_links[2:]:
#     driver = webdriver.Chrome('../../chromedriver', chrome_options=chrome_options)
    print('Make name:', make_name)
    while True:
        driver.get(make_link)
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='cc']/div[6]/div/ul/li")))
            break
        except:
            print('Trying to load the page again')
    all_cars = driver.find_elements_by_xpath("//*[@id='cc']/div[6]/div/ul/li")
    print('Web page is opened')
    
    dir_make = os.path.join('Cars', make_name)
    if not os.path.exists(dir_make):
        os.mkdir(dir_make)
        
    car_model_links = []
    for car in all_cars:
        if int(car.text[:4]) >= 2020:
            car_model_links.append(car.find_element_by_xpath(".//a").get_attribute('href'))

    print('Links to models:', car_model_links)
    print('Number of models:', len(car_model_links))

    for car_model_link in car_model_links:
        while True:
            driver.get(car_model_link)
            try:
                name = WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.XPATH, "//*[@id='cc']/article/div[1]/h1/strong"))).text[len(make_name) + 1:]
                print('Name of car model is found')
                break
            except:
                print('Wait Timeout for the name of car model')
                print(car_model_link)
                continue
        print(name)
        dir_name_model = os.path.join(dir_make, name)
        if not os.path.exists(dir_name_model):
            os.mkdir(dir_name_model)

        all_images = driver.find_elements_by_xpath("//*[contains(@id, 'thpg')]/div")

        links_to_image = []
        for n, image in enumerate(all_images):
            try:
                link_to_image = image.find_element_by_xpath(f".//*[contains(text(), '1024')]").get_attribute('href')
                links_to_image.append(link_to_image)
            except:
                print('Element has no 1024 resolution option:', f'picture number {n}')
                continue

        image_links = []
        for link_to_image in links_to_image:
            year = '2020' if '2020' in name else '2021'
            image_links.append(modify_link('https://www.netcarshow.com/' + '_'.join(make_name.split(' ')) + '-' + '_'.join(name.split(' ')[:-1]) + '-' + year + '-1024-' + link_to_image[-6:-4] + '.jpg'))
        print('Number of images:', len(image_links))
        
        
        
        def download_file(url, n):
            for _ in range(20):
                try:
                    time.sleep(3)
                    html = requests.get(url, proxies=proxyDict, stream=True)
                    image = np.array(Image.open(io.BytesIO(html.content)).resize((256, 256)))
                    image = np.expand_dims(image, 0)
                    pred = model.predict_classes(image)[0][0]
                    if pred == 1:
                        print('Request got noisy image')
                        continue
                    print(f'Found data for {n} picture:', url)
                    break
                except:
                    pass
            else:
                print('---Did not get an image for this link:', url)
                try:
                    return (html.content, n, url.split('/')[-1])
                except:
                    return (None, n, None)
            return (html.content, n, url.split('/')[-1])
        processes = []
        with ThreadPoolExecutor(max_workers=(10)) as executor:
            for n, url in enumerate(image_links):
                processes.append(executor.submit(download_file, url, n))
        final_results = []
        for task in as_completed(processes):
            try:
                final_results.append(task.result())
            except:
                print('Did not get some image')
                continue

        for picture, n, image_name in final_results:
            if picture == None:
                print(f'Image number {n} was not included because w')
                continue
            with open(os.path.join(dir_name_model, image_name), 'wb') as handler:
                handler.write(picture)

        print('-' * 20, f'Done with car model {name}', '-' * 20)
#     driver.close()
    print('**' * 20, f'Done with car make {make_name}', '**' * 20)

/Users/Arshak/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


Wait Timeout for Innitial page, trying again.
Innitial page is found


/Users/Arshak/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: use options instead of chrome_options


Make name: Acura
Web page is opened
Links to models: ['https://www.netcarshow.com/acura/2020-tlx_pmc_edition/', 'https://www.netcarshow.com/acura/2020-nsx/', 'https://www.netcarshow.com/acura/2020-mdx_pmc_edition/']
Number of models: 3
Wait Timeout for the name of car model
https://www.netcarshow.com/acura/2020-tlx_pmc_edition/
Name of car model is found
TLX PMC Edition (2020)
Element has no 1024 resolution option: picture number 20
Number of images: 20
Request got noisy image
Request got noisy image
Request got noisy image
Found data for 8 picture: https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-09.jpg
Request got noisy image
Found data for 0 picture: https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-01.jpg
Found data for 7 picture: https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-08.jpg
Found data for 6 picture: https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-07.jpg
Request got noisy image
Request got noisy image
Request got noisy image
Found 

Found data for 15 picture: https://www.netcarshow.com/Alfa_Romeo-Stelvio-2020-1024-0e.jpg
Request got noisy image
Request got noisy image
Found data for 17 picture: https://www.netcarshow.com/Alfa_Romeo-Stelvio-2020-1024-10.jpg
Request got noisy image
Request got noisy image
Found data for 14 picture: https://www.netcarshow.com/Alfa_Romeo-Stelvio-2020-1024-0d.jpg
Request got noisy image
Found data for 0 picture: https://www.netcarshow.com/Alfa_Romeo-Stelvio-2020-1024-01.jpg
Found data for 13 picture: https://www.netcarshow.com/Alfa_Romeo-Stelvio-2020-1024-0c.jpg
Found data for 8 picture: https://www.netcarshow.com/Alfa_Romeo-Stelvio-2020-1024-07.jpg
Found data for 12 picture: https://www.netcarshow.com/Alfa_Romeo-Stelvio-2020-1024-0b.jpg
Request got noisy image
Found data for 2 picture: https://www.netcarshow.com/Alfa_Romeo-Stelvio-2020-1024-0f.jpg
Found data for 4 picture: https://www.netcarshow.com/Alfa_Romeo-Stelvio-2020-1024-03.jpg
-------------------- Done with car model Stelvio (

...

Found data for 148 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-93.jpg
Request got noisy image
Request got noisy image
Found data for 155 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-9a.jpg
Found data for 153 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-98.jpg
Request got noisy image
Found data for 157 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-9c.jpg
Found data for 154 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-99.jpg
Request got noisy image
Found data for 159 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-9e.jpg
Request got noisy image
Request got noisy image
Request got noisy image
Request got noisy image
Found data for 158 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-9d.jpg
Found data for 162 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-a1.jpg
Request got noisy image
Request got noisy image
Found data for 161 picture: https://www.netcarshow.com/Audi-A5_Coupe-202

Found data for 235 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-ea.jpg
Found data for 234 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-e9.jpg
Request got noisy image
Found data for 240 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-ef.jpg
Found data for 243 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-f2.jpg
Request got noisy image
Request got noisy image
Request got noisy image
Found data for 244 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-f3.jpg
Found data for 242 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-f1.jpg
Request got noisy image
Request got noisy image
Found data for 237 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-ec.jpg
Request got noisy image
Request got noisy image
Found data for 246 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-f5.jpg
Found data for 251 picture: https://www.netcarshow.com/Audi-A5_Coupe-2020-1024-fa.jpg
Found data for 248 picture: https:

Request got noisy image
Found data for 62 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-3d.jpg
Request got noisy image
Request got noisy image
Request got noisy image
Found data for 69 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-44.jpg
Found data for 68 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-43.jpg
Request got noisy image
Found data for 71 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-46.jpg
Request got noisy image
Request got noisy image
Found data for 66 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-41.jpg
Found data for 67 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-42.jpg
Found data for 72 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-47.jpg
Found data for 73 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-48.jpg
Found data for 59 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-3a.jpg
Request got noisy image
Request

Request got noisy image
Request got noisy image
Found data for 143 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-8e.jpg
Request got noisy image
Found data for 149 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-94.jpg
Found data for 148 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-93.jpg
Request got noisy image
Found data for 150 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-95.jpg
Found data for 146 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-91.jpg
Request got noisy image
Request got noisy image
Request got noisy image
Request got noisy image
Found data for 151 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-96.jpg
Found data for 153 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-98.jpg
Request got noisy image
Request got noisy image
Found data for 155 picture: https://www.netcarshow.com/Audi-A5_Cabriolet-2020-1024-9a.jpg
Request got noisy image
Found data for 1

KeyboardInterrupt: 

In [5]:
image_links = ['https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-01.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-02.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-03.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-04.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-05.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-06.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-07.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-08.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-09.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-0a.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-0b.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-0c.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-0d.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-0e.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-0f.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-10.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-11.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-12.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-13.jpg',
 'https://www.netcarshow.com/Acura-TLX_PMC_Edition-2020-1024-14.jpg']

In [6]:
http_proxy  = "http://209.205.212.34:1203"
https_proxy = "https://209.205.212.34:1203"
ftp_proxy   = "ftp://209.205.212.34:1203"

proxyDict = { 
              "http"  : http_proxy, 
              "https" : https_proxy, 
              "ftp"   : ftp_proxy
            }

In [16]:
html = requests.get(image_links[0], proxies=proxyDict, stream=True)

In [17]:
image = np.array(Image.open(io.BytesIO(html.content)).resize((256, 256)))

In [18]:
image = np.expand_dims(image, 0)

In [19]:
model.predict_classes(image)[0][0]

1

In [12]:
def download_file(url, n, user_agent):
    headers = {
        'user-agent': user_agent}

    for _ in range(20):
        try:
            html = requests.get(url, proxies=proxyDict, stream=True, headers=headers)
            print(html.response)
            image = np.array(Image.open(io.BytesIO(html.content)).resize((256, 256)))
            image = np.expand_dims(image, 0)
            pred = model.predict_classes(image)[0][0]
            if pred == 1:
                print('Request got noisy image')
                continue
            print(f'Found data for {n} picture:', url)
            break
        except:
            print('Noooooo')
            pass
    return (html.content, n, url.split('/')[-1])
print('here')
processes = []
with ThreadPoolExecutor(max_workers=(10)) as executor:
    for n, url in enumerate(image_links):
        n_user_agent = random.randint(0, len(user_agents) - 1)
        processes.append(executor.submit(download_file, url, n, user_agents[n_user_agent]))
final_results = []
for task in as_completed(processes):
    try:
        final_results.append(task.result())
    except:
        print('Did not get some image')
        continue

for picture, n, image_name in final_results:
    with open(os.path.join(dir_name_model, image_name), 'wb') as handler:
                handler.write(picture)

NameError: name 'html' is not defined

Noooooo
Noooooo
Noooooo
Noooooo
Noooooo
Noooooo
Noooooo
Noooooo
Noooooo


## ML algorithm

In [1]:
from PIL import Image
from numpy import asarray
import numpy as np
import os

In [ ]:
noise_dir = 'ml_cars/noise/'
noise_imgs = os.listdir(noise_dir)
noise = []
for noise_img in noise_imgs:
    image = Image.open(noise_dir + noise_img).resize((256, 256))
    noise.append(asarray(image) / 255.)
# summarize shape
noise = np.array(noise)
noise_lables = np.ones(noise.shape[0])
print(noise.shape)

In [ ]:
cars_dir = 'ml_cars/cars/'
cars_imgs = os.listdir(cars_dir)
cars = []
for cars_img in cars_imgs:
    image = Image.open(cars_dir + cars_img).resize((256, 256))
    cars.append(asarray(image) / 255.)
# summarize shape
cars = np.array(cars)
cars_lables = np.zeros(cars.shape[0])
print(cars.shape)

In [ ]:
concat_img = np.concatenate((noise, cars), axis=0)
concat_lables = np.concatenate((noise_lables, cars_lables), axis=0)

In [ ]:
ind = np.arange(concat_img.shape[0])
np.random.shuffle(ind)

concat_img = concat_img[ind]
concat_lables = concat_lables[ind]

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential

classifier = Sequential()
classifier.add(Conv2D(16, (3, 3), input_shape=(256, 256, 3),
               activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(128, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(128, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())

classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=1, activation='sigmoid'))

classifier.compile(optimizer='adam', loss='binary_crossentropy',
                   metrics=['accuracy'])
classifier.summary()

In [ ]:
classifier.fit(concat_img, concat_lables, epochs=14)

In [ ]:
classifier.save('noise_classifier.h5')